In [ ]:
import sys
import os
scratch_dir = "/home/hice1/eresmini3/scratch"
repo_folder = "cold_diffusion_roi"
sys.path.append(f"{scratch_dir}/{repo_folder}")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from torch.utils.data import DataLoader

from utils.mri_data import SliceDataset
from utils.data_transform import DataTransform_Diffusion
from utils.sample_mask import RandomMaskGaussianDiffusion, RandomMaskDiffusion, RandomMaskDiffusion2D
from utils.misc import *
from help_func import print_var_detail

from diffusion.kspace_diffusion_2 import KspaceDiffusion
from utils.diffusion_train_parallel import Trainer # diffusion_train
from net.u_net_diffusion import Unet

print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

2.6.0+cu126
device: cuda:0


In [3]:
# ****** TRAINING SETTINGS ******
# dataset settings
acc = 8  # acceleration factor
frac_c = 0.04  # center fraction
path_dir_train = f"{scratch_dir}/data/singlecoil_train"
path_dir_test = f"{scratch_dir}/data/singlecoil_test"
img_mode = 'fastmri'  # 'fastmri' or 'B1000'
bhsz = 6
img_size = 320

In [4]:
# ====== Construct dataset ======
# initialize mask
mask_func = RandomMaskDiffusion(
    acceleration=acc,
    center_fraction=frac_c,
    size=(1, img_size, img_size),
)

# initialize dataset
data_transform = DataTransform_Diffusion(
    mask_func,
    img_size=img_size,
    combine_coil=True,
    flag_singlecoil=True,
)

# training set
dataset_train = SliceDataset(
    root=pathlib.Path(path_dir_train),
    transform=data_transform,
    sample_rate=0.1,
    challenge='singlecoil',
    num_skip_slice=5,
)

# test set
dataset_test = SliceDataset(
    root=pathlib.Path(path_dir_test),
    transform=data_transform,
    sample_rate=0.1,
    challenge='singlecoil',
    num_skip_slice=5,
)

dataloader_train = DataLoader(dataset_train, batch_size=bhsz, shuffle=True, worker_init_fn=lambda worker_id: np.random.seed(42))
dataloader_test = DataLoader(dataset_test, batch_size=bhsz, shuffle=True, worker_init_fn=lambda worker_id: np.random.seed(42))
print('len dataloader train:', len(dataloader_train))
print('len dataloader test:', len(dataloader_test))

len dataloader train: 433
len dataloader test: 49


In [5]:
# model settings
CH_MID = 64
# training settings
NUM_EPOCH = 50
learning_rate = 2e-5
time_steps = 1000
train_steps = NUM_EPOCH * len(dataloader_train)
train_steps = max(train_steps, 35000) #reduced from 700000
print('train_steps:',train_steps)
# save settings
absolute_path = f"{scratch_dir}/{repo_folder}"
PATH_MODEL = f'{absolute_path}/saved_models/fastmri_knee/FINAL_tuned_ROI_diffusion_'+str(img_mode)+'_'+str(acc)+'x_T'+str(time_steps)+'_S'+str(train_steps)+'/'
create_path(PATH_MODEL)

train_steps: 35000
The new directory is created!


In [8]:
# construct diffusion model
save_folder=PATH_MODEL
load_path=None #f"{save_folder}model_31400.pt"
blur_routine='Constant'
train_routine='Final'
sampling_routine='x0_step_down'
discrete=False

model = Unet(
    dim=CH_MID,
    dim_mults=(1, 2, 4, 8),
    channels=2,
).cuda()
print('model size: %.3f MB' % (calc_model_size(model)))

diffusion = KspaceDiffusion(
    model,
    image_size=img_size,
    device_of_kernel='cuda',
    channels=2,
    timesteps=time_steps,  # number of steps
    loss_type='l1',  # L1 or L2
    blur_routine=blur_routine,
    train_routine=train_routine,
    sampling_routine=sampling_routine,
    discrete=discrete,

    alpha=1.5,
    sobel_threshold=0.3,
    dilation_size=9,
).cuda()

Is Time embed used ?  True
model size: 53.992 MB


In [ ]:
# construct trainer and train

trainer = Trainer(
    diffusion,
    image_size=img_size,
    train_batch_size=bhsz,
    train_lr=learning_rate,
    train_num_steps=train_steps,  # total training steps
    gradient_accumulate_every=2,  # gradient accumulation steps
    ema_decay=0.995,  # exponential moving average decay
    fp16=False,  # turn on mixed precision training with apex
    save_and_sample_every=200,
    results_folder=save_folder,
    load_path=load_path,
    dataloader_train=dataloader_train,
    dataloader_test=dataloader_test,
)
trainer.train()

LOSS:   0%|                                                                                                                                                                                                   | 0/35001 [00:00<?, ?it/s]

Starting training from step 0
GPU Memory before training: 869.41 MB
Step 0
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])


Loss=0.224139:   0%|                                                                                                                                                                              | 1/35001 [00:23<230:33:27, 23.71s/it]

Step 1
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])


Loss=0.180832:   0%|                                                                                                                                                                              | 2/35001 [00:25<104:16:44, 10.73s/it]

...

Step 569
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])


Loss=0.048694:   2%|██▊                                                                                                                                                                          | 570/35001 [18:12<17:40:07,  1.85s/it]

Step 570
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])


Loss=0.053978:   2%|██▊                                                                                                                                                                          | 571/35001 [18:14<17:47:07,  1.86s/it]

Step 571
create_roi_mask: mask.shape=torch.Size([6, 1, 320, 320, 2])
